## 問題1 ブレンディングのスクラッチ実装

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.linear_model import PassiveAggressiveRegressor, ARDRegression, RidgeCV
from sklearn.linear_model import TheilSenRegressor, RANSACRegressor, HuberRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor, VotingRegressor, StackingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


train = pd.read_csv('train.csv')

x_df = train.loc[:,["GrLivArea", "YearBuilt"]]
y_df = train.loc[:, 'SalePrice']
# 数値を底を０に調整
x_df = x_df.apply(np.log1p)
y_df = y_df.apply(np.log1p)

x_train, x_test, y_train, y_test = train_test_split(x_df.values, y_df.values, test_size=0.2, random_state=42)

In [398]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

# 学習に使いたいデータセットをリストに格納
data_set = []
data_set.append((x_train, x_test, y_train, y_test))
# SVRのハイパーパラメーター（変更しやすいように別で辞書で用意）

def blending(data_set, c=10, g=1, ep=0, Kernel="rbf"):
    """
    訓練用のデータセットをリストで入力、MSEを返す
    ----------
    Parameter
    data_set : list
      学習・検証するデータ（4つ）
    c
      SVMのハイパーパラメーター
    ----------
    Return
    mse_list : list
      学習したそれぞれのmseをリストで返す
    """
    learn_set = [LinearRegression(), 
                 SVR(C=c, gamma=g, kernel=Kernel, epsilon=ep), 
                 DecisionTreeRegressor(random_state=0)]
    clf_list = []
    for learn in learn_set:
        clf = learn
        clf_list.append((clf))

    pred_list = np.array([])
    mse_list = np.array([])
    score_list = np.array([])

    for x_train, x_test, y_train, y_test in data_set:
        for i, model in enumerate(clf_list):
            model.fit(x_train, y_train)
            pred = model.predict(x_test)
            score = model.score(x_test, y_test)
            pred_list = np.append(pred_list, pred)
            score_list = np.append(score_list, f'score :{model.__class__.__name__} : {score}')

            mse = mean_squared_error(y_test, pred)
            mse_list = np.append(mse_list, f'mse : {model.__class__.__name__} : {mse}')
    return mse_list, pred_list, score_list

# MSEの結果
# SVMのカーネルをrbfとlinearで比較 
# linearの場合は半分以上mseの数値が下がった
print(blending(data_set, c=100, g=10, ep=0, Kernel="linear")[0])
print('------------------------------------')
print(blending(data_set, c=100, g=10, ep=0, Kernel="linear")[2])

pred_1 = model.predict(x_test)
pred_2 = model.predict(x_test)
pred_3 = model.predict(x_test)

LRclf = LinearRegression()
SVR_clf = SVR(C=100, gamma=10, kernel="linear", epsilon=0)
Tree_clf = DecisionTreeRegressor(random_state=0)

LRclf.fit(x_train, y_train)
SVR_clf.fit(x_train, y_train)
Tree_clf.fit(x_train, y_train)

LR_pred = LRclf.predict(x_test)
SVR_pred = SVR_clf.predict(x_test)
Tree_pred = Tree_clf.predict(x_test)

# 3行292列 それぞれの学習データを行報告に追加する
total_pred = np.concatenate([[LR_pred, SVR_pred, Tree_pred]], axis=1)
# それぞれの列で平均値を出力する
pred_mean = np.mean(total_pred, axis=0)

# mseを計算
blending_mse = mean_squared_error(y_test,pred_mean)
print(f'mse : blending_Data : {blending_mse}')

# MSEが下がった手法
# パターン１ それぞれの平均をとった（線形回帰・SVM・決定木）
# パターン２ 対数変換を行なった（前処理）
# パターン３ SVMのカーネルをrbf→linearに変更した


['mse : LinearRegression : 0.05207835114334287'
 'mse : SVR : 0.05393751032686638'
 'mse : DecisionTreeRegressor : 0.06602628200076659']
------------------------------------
['score :LinearRegression : 0.7209255921255862'
 'score :SVR : 0.7109628391409917'
 'score :DecisionTreeRegressor : 0.6461822398563322']
mse : blending_Data : 0.047431510278425515


## 【問題2】バギングのスクラッチ実装

バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

In [396]:

LRclf = LinearRegression()
SVR_clf = SVR(C=100, gamma=10, kernel="linear", epsilon=0)
Tree_clf = DecisionTreeRegressor(random_state=0)

LRclf.fit(x_train, y_train)
SVR_clf.fit(x_train, y_train)
Tree_clf.fit(x_train, y_train)

LR_pred = LRclf.predict(x_test)
SVR_pred = SVR_clf.predict(x_test)
Tree_pred = Tree_clf.predict(x_test)

# 3行292列 それぞれの学習データを行報告に追加する
total_pred = np.concatenate([[LR_pred, SVR_pred, Tree_pred]], axis=1)
# それぞれの列で平均値を出力する
pred_mean = np.mean(total_pred, axis=0)

# mseを計算
blending_mse = mean_squared_error(y_test,pred_mean)
# print(f'mse : blending_Data : {blending_mse}')

# ベースの推定
print('バギングのベースとなるmse')
print(f'線形回帰 : {mean_squared_error(y_test,LR_pred)}')
print(f'SVM      : {mean_squared_error(y_test,SVR_pred)}')
print(f'決定木   : {mean_squared_error(y_test,Tree_pred)}')

# dataを作り直す回数
n = 20

models = []

for i in range(n):
    # 訓練データをさらにランダムに分割
    X_bagging, _X, y_bagging, _y = train_test_split(x_train, y_train)
    # モデルの定義
    model = LinearRegression()
    # 学習
    model.fit(X_bagging,y_bagging)
    # モデルを記録
    models.append(model)

# 初期配列
prediction = np.zeros(len(x_test))

# ランダム分割によって学習したモデルで推定結果を出す
for model in models:
    _predict = model.predict(x_test)
    prediction = np.append(prediction, _predict)#０配列の列方向に追加していく

# それぞれの列で平均値を出力する
prediction_bagging = prediction.reshape(21, 292)#列を整理する

pred_mean_1 = np.mean(prediction_bagging, axis=0)#列（縦方向）の平均
bagging_mse = mean_squared_error(y_test,pred_mean_1)#MSE
print(f'mse : bagging_mse : {bagging_mse}')

# 20回ランダムに分割を繰り返したデータの平均をとると大きくMSEは下がった


バギングのベースとなるmse
線形回帰 : 0.05207835114334287
SVM      : 0.05393751032686638
決定木   : 0.06602628200076659
mse : bagging_mse : 0.36997561255868244


## 【問題3】スタッキングのスクラッチ実装

In [395]:
class Stacking():
    """
    スタッキング
    """
    def __init__(self, max_depth, splits, models):
        """コンストラクタ
        Parameters
        ----------
        max_depth : スタッキングの最大深さ
        splits : データ分割数
        models : 使用モデル一覧
        fit_models : 学習済みモデル保存用リスト
        """
        self.max_depth = max_depth
        self.n_splits = splits
        self.models = models
        self.fit_models = []
        
    def calc_blending(self,x,y,model):
        """ブレンドデータ生成
        Parameters
        ----------
        x : 説明変数
        y : 目的変数
        m : モデル
        """
        # blendingの初期化
        blending = np.zeros(len(x))
        
        # データの分割
        kf = KFold(n_splits=self.n_splits, shuffle=False)
 
        # 分割されたインデックスでループ
        for train_index, test_index in kf.split(x):
            # データ分割
            x_train, x_test = x[train_index], x[test_index]
            y_train, y_test = y[train_index], y[test_index]
            y_train = y_train.ravel()
            y_test = y_test.ravel()

            # 学習
            model.fit(x_train, y_train)
            # 学習済みモデルの保存
            self.fit_models.append(model)
            # テストデータのインデックスに予測値を格納
            blending[test_index] = model.predict(x_test)

        return blending

    print(blending)
    
    def fit(self,x,y,depth):
        """該当深さの学習実行
        Parameters
        ----------
        x : 説明変数
        y : 目的変数
        depth : スタッキングの「現在の」深さ
        """
        # スタッキングの「現在の」深さをメンバ変数化
        self.depth=depth
        
        # 最大深さまで到達していれば、その深さのモデルを学習させて処理終了
        if self.depth == self.max_depth:
            # 当該深さでのモデルを取得
            self.model = self.models[self.depth]
            # 学習
            self.model.fit(x,y)
            return
        
        # 当該深さでのモデルを取得
        models = self.models[self.depth]
        self.blending = np.zeros([len(x),len(models)])
        #1168*3
        # print(self.blending.shape)
        
        # この階層の全てのモデルで学習
        for i,model in enumerate(models):
            _blending = self.calc_blending(x, y, model)
            # print(_blending.shape)
            self.blending[:,i] = _blending
            # xxxxxxxxXXXXXXXXXXXXXXXXXXXXXX = _blending

        # 再帰学習
        # コンストラクタ生成の際の引数は同じ
        self.stk = Stacking(self.max_depth, self.n_splits, self.models)
        
        # 学習実行の際は、ブレンドデータを説明変数として渡す。深さも1加えてやる
        self.stk.fit(self.blending,y,depth+1)
        
    def predict(self,x):
        """予測
        Parameters
        ----------
        x : 説明変数
        y : 目的変数
        depth : スタッキングの「現在の」深さ
        """
        # 最大深さの場合は最終的な予測値を出力するのみ
        if self.depth == self.max_depth:
            # 予測
            prediction = self.model.predict(x)
            # 返す
            return prediction
        # 最大深さに達していない場合は、再帰的に呼び出す
        else:
            
            # 予測値を0で初期化
            self.prediction = np.zeros(len(x))
            # 次の階層に渡すブレンドデータ(仮)の作成
            self.blending = np.zeros([len(x),len(self.models[self.depth])])
            # この階層の学習済みモデルでループ
            count = 0 # 現在どのモデルを回しているか把握
            for model in self.fit_models:
                # 予測し、0で初期化している予測値に加算
                self.prediction += model.predict(x)
                # 1種類のモデル学習の終了判定
                count+=1
                if count%self.n_splits == 0:
                    # 予測値を加算してきたので割って平均値を算出
                    self.prediction = self.prediction/self.n_splits
                    # その平均値を次の階層で使用する説明変数に格納
                    self.blending[:,int(count/self.n_splits)-1] = self.prediction
                    # 次の種類のモデルでの予測のため、予測値は初期化しておく
                    self.prediction = np.zeros(len(x))
            # 次の階層の予測関数
            prediction = self.stk.predict(self.blending)
            return prediction

models = {
    0:[LinearRegression(),DecisionTreeRegressor(),RandomForestRegressor()],
    1:[ARDRegression(),SGDRegressor(),DecisionTreeRegressor()],
    2:[HuberRegressor(),ARDRegression(),RandomForestRegressor()],
    3:LinearRegression()
}

stk = Stacking(max_depth=3,splits=5,models=models)
stk.fit(x_train,y_train,0)

stk_prediction = stk.predict(x_test)
# print(stk_prediction)
stk_mse = mean_squared_error(y_test,stk_prediction)#MSE
print(f'mse : stk_mse : {stk_mse}')


<function blending at 0x7fbf19f47ca0>
mse : stk_mse : 0.046503593119692065


In [ ]:
# スタッキングも単独のモデルよりは精度が上がったが、バギングの方が精度の上昇が大きかった